# ADA Homework 2 - Data from the Web

In [103]:
# Import libraries
import requests
import json
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

## 1. 200 QS Top Universities from www.topuniversities.com 

In [104]:
NB_BEST = 200
QS_URL = "https://www.topuniversities.com"
QS_RANK_URL = "https://www.topuniversities.com/university-rankings/world-university-rankings/2018"
QS_DATA_URL = "https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt"

In [105]:
QS_data_req = requests.get(QS_DATA_URL)
QS_data = QS_data_req.json()

In [106]:
def get_QS_number(type_string):
    wrapper = uni_soup.find("div", class_=type_string)
    if wrapper:
        string = wrapper.find("div", class_="number").text[1:]
        return int(re.sub("[^0-9^.]", "", string))
    else:
        return np.nan

In [107]:
QS_ranking = pd.DataFrame(columns = ["nid", "title", "rank_display", "country", "region", "score"])

for i in range(NB_BEST):
    df_i = pd.DataFrame.from_dict(QS_data["data"][i], orient='index').T
    df_i = df_i.drop(["stars", "guide", "cc", "logo", "core_id", "url"], axis=1)
    
    # Get linked site with more information
    uni_url = QS_URL + QS_data["data"][i]["url"]
    uni_request = requests.get(uni_url)
    uni_page_body = uni_request.text
    uni_soup = BeautifulSoup(uni_page_body, 'html.parser')
    
    # Get total number of students
    df_i["total_students"] = get_QS_number("total student")
    # Get number of international students
    df_i["international_students"] = get_QS_number("total inter")
    # Get number of faculty members
    df_i["faculty_members"] = get_QS_number("total faculty")
    # Get number of international faculty members
    df_i["inter_faculty_members"] = get_QS_number("inter faculty")
    
    QS_ranking = QS_ranking.append(df_i)
    
    
QS_ranking = QS_ranking.add_suffix("_QS")
QS_ranking = QS_ranking.rename(columns = {'title_QS':'name'})
QS_ranking = QS_ranking.set_index("name")


# Cannot convert columns to int, because NaN's can only exist in float or object columns
#QS_ranking.total_students = QS_ranking.total_students.astype(int)
#QS_ranking.international_students = QS_ranking.international_students.astype(int)
#QS_ranking.faculty_members = QS_ranking.faculty_members.astype(int)
#QS_ranking.inter_faculty_members = QS_ranking.inter_faculty_members.astype(int)

In [108]:
QS_ranking.head()

,country_QS,faculty_members_QS,inter_faculty_members_QS,international_students_QS,nid_QS,rank_display_QS,region_QS,score_QS,total_students_QS
name,,,,,,,,,
Massachusetts Institute of Technology (MIT),United States,2982.0,1679.0,3717.0,294850,1,North America,100,11067.0
Stanford University,United States,4285.0,2042.0,3611.0,297282,2,North America,98.7,15878.0
Harvard University,United States,4350.0,1311.0,5266.0,294270,3,North America,98.4,22429.0
California Institute of Technology (Caltech),United States,953.0,350.0,647.0,294562,4,North America,97.7,2255.0
University of Cambridge,United Kingdom,5490.0,2278.0,6699.0,294561,5,Europe,95.6,18770.0


In [109]:
QS_ranking.describe()

,faculty_members_QS,inter_faculty_members_QS,international_students_QS,total_students_QS
count,199.000000,198.000000,199.000000,199.000000
mean,2912.949749,786.409091,5078.261307,25709.130653
std,2010.741417,666.690652,3924.655224,17135.556845
min,86.000000,1.000000,47.000000,532.000000
25%,1636.500000,327.000000,2552.500000,15140.500000
50%,2605.000000,599.500000,4250.000000,23775.000000
75%,3633.500000,1027.750000,6880.500000,32216.000000
max,16421.000000,3905.000000,27109.000000,141939.000000


In [110]:
QS_ranking.dtypes

country_QS                    object
faculty_members_QS           float64
inter_faculty_members_QS     float64
international_students_QS    float64
nid_QS                        object
rank_display_QS               object
region_QS                     object
score_QS                      object
total_students_QS            float64
dtype: object

## 2. 200 TH Top Universities from www.timeshighereducation.com

In [111]:
TH_URL = "http://timeshighereducation.com"
TH_RANK_URL = "http://timeshighereducation.com/world-university-rankings/2018/world-ranking"
TH_DATA_URL = "https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json"

In [112]:
TH_data_req = requests.get(TH_DATA_URL)
TH_data = TH_data_req.json()

In [114]:
TH_ranking = pd.DataFrame(columns = ["nid", "name", "rank", "scores_overall", "location"])

for i in range(NB_BEST):
    df_i = pd.DataFrame.from_dict(TH_data["data"][i], orient='index').T
    drop_cols = df_i.columns.difference(["nid", "name", "rank", "scores_overall", "location"])
    df_i = df_i.drop(drop_cols, axis=1)
    
    df_i["total_students"] = int(re.sub("[^0-9^.]", "", TH_data["data"][i]["stats_number_students"]))
    percentage_intl = int(re.sub("[^0-9^.]", "", TH_data["data"][i]["stats_pc_intl_students"]))
    df_i["international_students"] = int(df_i.total_students * percentage_intl / 100)
    
    TH_ranking = TH_ranking.append(df_i)
    
TH_ranking = TH_ranking.add_suffix("_TH")
TH_ranking = TH_ranking.rename(columns = {'name_TH':'name'})
TH_ranking = TH_ranking.set_index("name")
TH_ranking.total_students = TH_ranking.total_students_TH.astype(int)
TH_ranking.international_students = TH_ranking.international_students_TH.astype(int)

In [115]:
TH_ranking.head()

,international_students_TH,location_TH,nid_TH,rank_TH,scores_overall_TH,total_students_TH
name,,,,,,
University of Oxford,7755.0,United Kingdom,468,1,94.3,20409.0
University of Cambridge,6436.0,United Kingdom,470,2,93.2,18389.0
California Institute of Technology,596.0,United States,128779,=3,93.0,2209.0
Stanford University,3485.0,United States,467,=3,93.0,15845.0
Massachusetts Institute of Technology,3800.0,United States,471,5,92.5,11177.0


In [116]:
TH_ranking.describe()

,international_students_TH,total_students_TH
count,200.000000,200.00000
mean,4964.715000,24746.29500
std,3108.431593,13353.01984
min,36.000000,515.00000
25%,2789.000000,14880.25000
50%,4437.500000,23130.50000
75%,6693.500000,32933.25000
max,16846.000000,69427.00000


In [117]:
TH_ranking.dtypes

international_students_TH    float64
location_TH                   object
nid_TH                        object
rank_TH                       object
scores_overall_TH             object
total_students_TH            float64
dtype: object

In [118]:
import difflib

In [122]:
def match_up(x, indexes):
    matches = difflib.get_close_matches(x, indexes)
    if matches:
        return matches[0]
    else:
        return x

QS_ranking.index = QS_ranking.index.map(lambda x: match_up(x, TH_ranking.index))
merged = QS_ranking.join(TH_ranking, how='outer')
merged.head()

,country_QS,faculty_members_QS,inter_faculty_members_QS,international_students_QS,nid_QS,rank_display_QS,region_QS,score_QS,total_students_QS,international_students_TH,location_TH,nid_TH,rank_TH,scores_overall_TH,total_students_TH
name,,,,,,,,,,,,,,,
Aalto University,Finland,1257.0,370.0,1831.0,294263,=137,Europe,56,12147.0,2548.0,Finland,767,190,52.7,12744.0
Aarhus University,Denmark,2316.0,602.0,3762.0,294648,119,Europe,59.3,26226.0,3020.0,Denmark,589,=109,59.4,25167.0
Arizona State University,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8450.0,United States,591,=126,58.1,44475.0
Australian National University,Australia,1600.0,927.0,5551.0,294616,20,Oceania,87.1,14442.0,5595.0,Australia,502,48,71.6,15986.0
Autonomous University of Barcelona,Italy,2990.0,153.0,4195.0,294595,=188,Europe,49.1,63399.0,5169.0,Spain,666,=147,56.2,32309.0
